## <b><font color='darkblue'>Preface</font></b>
([source](https://cobusgreyling.medium.com/using-langchain-with-model-context-protocol-mcp-e89b87ee3c4c)) <b><font size='3ptx'>The [Model Context Protocol](https://en.wikipedia.org/wiki/Model_Context_Protocol) (MCP) is an open-source protocol developed by Anthropic, focusing on safe and interpretable Generative AI systems. MCP emerged from the need to address a key limitation with Large Language Model (LLM) applications, that being their isolation from external data sources and tools.</font></b>

<b>One of the key focus areas of LLM-based applications has been the aspect of Data Delivery</b>. Getting data to the LLM for inference, this has been the objective of RAG implementations, Fine-tuning and how also the objective of MCP.

MCP’s primary purpose is to standardise how LLM-based applications connect to diverse systems, as seen in the image below:
![MCP servers](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*cAZLUf-GB9jXmOHD_ThaEA.png)

## <b><font color='darkblue'>Eliminating Custom Integration</font></b>
<b><font size='3ptx'>There is this challenge with AI Agents to delivery data to the AI Agent, or in other words; integrate the AI Agent/LLM-based application to external data sources.</font></b>

There is been numerous attempts to integrate somehow seamless by leveraging GUI’s, web browser and web search. All of these avenues have advantages and disadvantages.
![ex](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*A4Sjk8hqd8TDEEyVaXahcw.gif)

<b>MCP has the potential to function as a universal interface, think of it as the virtual / software version of USB-C for AI.</b> Enabling seamless, secure and scalable data exchange between LLMs/AI Agents and external resources.

<b>MCP uses a client-server architecture where MCP hosts</b> (AI applications) <b>communicate with MCP servers</b> (data/tool providers). Developers can use MCP to build reusable, modular connectors, with pre-built servers available for popular platforms, creating a community-driven ecosystem.

MCP’s open-source nature encourages innovation, allowing developers to extend its capabilities while maintaining security through features like granular permissions. <b>Ultimately, MCP aims to transform AI Agents from isolated chatbots into context-aware, interoperable systems deeply integrated into digital environments</b>.

## <b><font color='darkblue'>Step by Step Instructions</font></b>
<b><font size='3ptx'>Anthropic’s Model Context Protocol (MCP) is an open source protocol to connect LLMs with context, tools, and prompts. It has a growing number of 𝘴𝘦𝘳𝘷𝘦𝘳𝘴 for connecting to various tools or data sources. Here, we show how to connect any MCP server to LangGraph agents & use MCP tools…</font></b>

If you are like me, getting a prototype work, no matter how simple it is, brings an immense sense of clarity and understanding; at leat in my own mind.

### <b><font color='darkgreen'>What is the Model Context Protocol (MCP)?</font></b>
([source](https://apidog.com/blog/fastmcp/)) <b><font size='3ptx'>Before diving deeper into FastMCP, it's essential to grasp the core concepts of MCP itself.</font></b>

MCP defines a standardized way for LLM applications (clients) to interact with external systems (servers). An MCP server can expose several key components:
* **Tools:** These are essentially functions that an LLM can request the server to execute. Think of them like POST endpoints in a traditional API. They perform actions, potentially interact with other systems (databases, APIs, hardware), and return results. For example, a tool could send an email, query a database, or perform a calculation.
* **Resources:** These expose data that an LLM can read or retrieve. Similar to GET endpoints, resources provide information to enrich the LLM's context. This could be anything from configuration files and user profiles to real-time data streams.
* **Prompts:** These are reusable templates for structuring interactions with the LLM. They help guide the conversation and ensure consistent outputs for specific tasks.
* **Context:** Servers can provide contextual information, including instructions on how to best interact with the available tools and resources.

<b>MCP aims to create a robust and secure ecosystem where LLMs can reliably access and utilize external capabilities</b>.

### <b><font color='darkgreen'>Why Choose FastMCP?</font></b>
<b><font size='3ptx'>While you could implement the MCP specification directly using lower-level SDKs, FastMCP offers compelling advantages, particularly for Python developers:</font></b>
* **🚀 Fast Development:** Its high-level interface significantly reduces the amount of code needed, accelerating the development process. Often, defining a tool or resource is as simple as decorating a standard Python function.
* **🍀 Simplicity:** FastMCP abstracts away the complex details of server setup, protocol handling, content types, and error management, minimizing boilerplate.
* **🐍 Pythonic:** Designed with Python best practices in mind, it feels natural and intuitive for developers familiar with the language, leveraging features like type hints and decorators.
* **🔍 Complete:** FastMCP aims to provide a comprehensive implementation of the core MCP specification, ensuring compatibility and access to the protocol's full potential.

FastMCP version 1 proved highly successful and is now integrated into the official MCP Python SDK. Version 2 builds upon this foundation, introducing advanced features focused on simplifying server interactions, such as flexible clients, server proxying, and composition patterns.

### <b><font color='darkgreen'>Virtual environment</font></b>
It is good practice to create a virtual environment to install and run code; the command below creates the virtual environment called `MCP_Demo`.

In [1]:
# !python3 -m venv MCP_Demo

Then run this command to activate (enter) the virtual environment:

In [2]:
#!source MCP_Demo/bin/activate

You will see your command prompt is updated with (`MCP_Demo`).

Then run the following lines of code in sequence to install the required packages:
```shell
$ pip install langchain-mcp-adapters
$ pip install langchain-mcp-adapters
$ export GEMINI_API_KEY=<your_api_key>
```

In [4]:
!pip freeze | grep 'langchain-mcp-adapters'

langchain-mcp-adapters==0.1.8


### <b><font color='darkgreen'>MCP Server implementation</font></b>
<b><font size='3ptx'>An LLM MCP (Model Control Plane) Server is a backend component that manages and routes requests to large language models.</font></b>

The MCP server acts as a gateway between user applications and the model infrastructure, ensuring efficient, secure, and scalable access to LLM capabilities. It's especially useful in systems that serve multiple models or need fine-grained control over inference workflows.

- **`math_server.py`**:

```python
from mcp.server.fastmcp import FastMCP

# name or identifier of the LLM app or service
mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
  """Add two numbers"""
  return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
  """Multiply two numbers"""
  return a * b

if __name__ == "__main__":
  # "stdio" tells the MCP server to read requests from standard input (stdin) and write responses to standard output (stdout).
  mcp.run(transport="stdio")
```

Run the server with the following command:
```shell
$ python math_server.py
```

While the MCP server is running in the one tab, let's open a new tab to develop MCP client.

### <b><font color='darkgreen'>MCP Client</font></b>
<b><font size='3ptx'>An LLM MCP Client is a component or library that communicates with an LLM MCP (Model Control Plane) Server to send prompts and receive model-generated responses.</font></b>

It abstracts the details of the transport layer (e.g., stdio, http, or websocket) and provides a simple interface for applications to interact with large language models. 

- **`mcp_client.py`**:

```python
# Create server parameters for stdio connection
import os
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import asyncio

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

working_directory = os.getcwd()
executing_path = Path(working_directory) / Path("math_server.py")

server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your math_server.py file
    args=[executing_path],
)

async def run_agent():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # Get tools
            tools = await load_mcp_tools(session)

            # Create and run the agent
            agent = create_react_agent(model, tools)
            agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
            return agent_response

# Run the async function
if __name__ == "__main__":
    result = asyncio.run(run_agent())
    for message in result['messages']:
      print(message.pretty_print())
      print('\n')
```

From above sample code, it leverage function [**`langchain.agents.react.agent.create_react_agent`**](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent) to bind the tools with LLM mode.

In [5]:
# Confirm we got correct execution path:
import os
from pathlib import Path

working_directory = os.getcwd()
executing_path = Path(working_directory) / Path("math_server.py")

print(executing_path)

/home/john/Gitrepos/ml_articles/medium/Using_Langchain_with_model-context-protocol_mcp/math_server.py


In [4]:
# Confirm our model API key setting is correct:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

Run the client with the command: `python mcp_client.py`

The client will run once and end with the output below:
```
================================ Human Message =================================

what's (3 + 5) x 12?
None


================================== Ai Message ==================================
Tool Calls:
  add (179d68f6-144c-4dcb-afd0-c8595deba9c6)
 Call ID: 179d68f6-144c-4dcb-afd0-c8595deba9c6
  Args:
    a: 3.0
    b: 5.0
None


================================= Tool Message =================================
Name: add

8
None


================================== Ai Message ==================================
Tool Calls:
  multiply (0f6dfdda-3830-463f-9cfe-2b7c5f084a64)
 Call ID: 0f6dfdda-3830-463f-9cfe-2b7c5f084a64
  Args:
    a: 8.0
    b: 12.0
None


================================= Tool Message =================================
Name: multiply

96
None


================================== Ai Message ==================================

(3 + 5) x 12 is 96.
None
```

MCP is a convenient way of integrating AI Agents with information and services supplying context and memory.

## <b><font color='darkblue'>Supplement</font></b>
* [Gemini API quickstart](https://ai.google.dev/gemini-api/docs/quickstart)
* [**LangChain doc:** ChatGoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/) 
* [A Beginner's Guide to Use FastMCP](https://apidog.com/blog/fastmcp/)